# CABH Data Monitoring System
This notebook implements the CABH data monitoring system for tracking sensor data points and generating reports.

## Setup and Configuration

In [ ]:
import pandas as pd
import json
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from datetime import datetime
import os
import pymysql
from pathlib import Path

In [ ]:
# Email configuration
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
SENDER_EMAIL = "abdullah@edsglobal.com"
SENDER_PASSWORD = "txam vysk plfl tdvk"  # Gmail App Password
RECIPIENT_EMAILS = [
    "ashish@edsglobal.com",
    "abdullah@edsglobal.com",
    "piyush@edsglobal.com",
    "gurneet@edsglobal.com",
    "lakshmy@edsglobal.com"
]

## Database Functions
Functions for connecting to the database and counting data points

In [ ]:
def count_data_points_batch(device_ids):
    # Implementation from count_device_data_db.py
    pass  # TODO: Implement database connection and query logic

## Data Processing Functions
Functions for processing monitor data and generating reports

In [ ]:
def process_monitor_list(monitors_data):
    # Parse the JSON data if it's a string
    if isinstance(monitors_data, str):
        monitors = json.loads(monitors_data)
    else:
        monitors = monitors_data
    
    # Access the Data array from the JSON structure
    monitors_list = monitors.get('Data', [])
    
    # Get all device IDs
    device_ids = [int(monitor["deviceID"]) for monitor in monitors_list]
    
    # Get counts for all devices in one batch
    data_points_dict = count_data_points_batch(device_ids)
    
    # Create results list using the batch data
    results = []
    for monitor in monitors_list:
        device_id = monitor["deviceID"]
        device_data = data_points_dict.get(device_id, {"count": 0, "last_timestamp": None})
        data_points = device_data["count"]
        data_percentage = round((data_points / 1440) * 100, 2)  # Calculate percentage
        result = {
            "deviceID": device_id,
            "deployementID": monitor["deployementID"],
            "typology": monitor["typology"],
            "address": monitor["address"],
            "data_points_24h": data_points,
            "data_percentage": data_percentage,
            "last_timestamp": device_data["last_timestamp"]
        }
        results.append(result)
    
    return results

## Email Functions
Functions for sending monitoring reports via email

In [ ]:
def send_monitoring_report(monitor_report_path, low_data_report_path):
    try:
        # Create message
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = ", ".join(RECIPIENT_EMAILS)
        msg['Subject'] = f"CABH Data Monitoring Report - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

        # Email body
        body = """Hello,

Please find attached the latest CABH data monitoring reports:
1. Complete Monitor Data Points Report
2. Low Data Sensors Report (devices with <95% data)

This is an automated email. Please do not reply.

Best regards,
Abdullah Kidwai"""
        msg.attach(MIMEText(body, 'plain'))

        # Attach monitor report
        with open(monitor_report_path, 'rb') as f:
            monitor_attachment = MIMEApplication(f.read(), _subtype='csv')
            monitor_attachment.add_header('Content-Disposition', 'attachment', 
                                        filename=os.path.basename(monitor_report_path))
            msg.attach(monitor_attachment)

        # Attach low data report
        with open(low_data_report_path, 'rb') as f:
            low_data_attachment = MIMEApplication(f.read(), _subtype='csv')
            low_data_attachment.add_header('Content-Disposition', 'attachment', 
                                         filename=os.path.basename(low_data_report_path))
            msg.attach(low_data_attachment)

        # Send email
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.send_message(msg)

        print("\nMonitoring reports sent successfully via email.")
        print(f"Reports sent to: {', '.join(RECIPIENT_EMAILS)}")

    except Exception as e:
        print(f"\nError sending email: {str(e)}")

## Generate Reports
Run the monitoring system and generate reports

In [ ]:
def generate_low_data_report():
    try:
        # First, run the monitor data points report
        with open('monitor_data.json', 'r') as file:
            monitor_data = json.load(file)
        
        # Process the monitor list and get results
        results = process_monitor_list(monitor_data)
        
        # Get current timestamp for filenames
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Create DataFrame and export to CSV with timestamp
        df = pd.DataFrame(results)
        output_filename = f'monitor_data_points_report_{timestamp}.csv'
        df.to_csv(output_filename, index=False)
        
        # Filter for low data sensors
        low_data_df = df[df['data_percentage'] < 95.0].sort_values('data_percentage')
        
        # Export low data sensors to a separate CSV file with timestamp
        low_data_output = f'low_data_sensors_report_{timestamp}.csv'
        low_data_df.to_csv(low_data_output, index=False)
        
        # Print summary
        print("\nMonitor Data Points Report Summary")
        print("-" * 80)
        print(f"Total sensors analyzed: {len(df)}")
        print(f"Sensors with low data (<95%): {len(low_data_df)}")
        print(f"Full report exported to {output_filename}")
        print(f"Low data sensors report exported to {low_data_output}")
        
        # Send reports via email
        send_monitoring_report(output_filename, low_data_output)
        
        if not low_data_df.empty:
            print("\nLow Data Sensors:")
            print(low_data_df[['deviceID', 'deployementID', 'data_percentage']].to_string(index=False))
        
    except FileNotFoundError:
        print("Error: monitor_data.json file not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

In [ ]:
# Run the report generation
generate_low_data_report()